In [1]:
# !pip install z3-solver

In [2]:
# !pip install snntorch

In [1]:
# imports
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools
import time

from z3 import *

In [2]:
# dataloader arguments
batch_size = 128
data_path='/data/mnist'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
  # Define a transform
transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

In [4]:
# temporary dataloader if MNIST service is unavailable
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

mnist_train = datasets.MNIST(root = './', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root = './', train=False, download=True, transform=transform)

'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'MNIST.tar.gz'


In [5]:
# Create DataLoaders
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

In [6]:
# Network Architecture
num_inputs = 28*28
num_hidden = [200,100,50]
num_outputs = 10

# Temporal Dynamics #Arka
num_steps = 25  
time_steps = 25
beta = 1

## Defining the Model

In [7]:
# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden[0],bias=False)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden[0], num_hidden[1],bias=False)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(num_hidden[1], num_hidden[2],bias=False)
        self.lif3 = snn.Leaky(beta=beta)
        self.fc4 = nn.Linear(num_hidden[2], num_outputs,bias=False)
        self.lif4 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        mem4 = self.lif4.init_leaky()

        # Record the final layer
        spk1_rec = []
        spk2_rec = []
        spk3_rec = []
        spk4_rec = []
        mem4_rec = []

        for step in range(num_steps):
            cur1 = self.fc1(x[step])
            spk1, mem1 = self.lif1(cur1, mem1)
            
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            
            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)
            
            cur4 = self.fc4(spk3)
            spk4, mem4 = self.lif4(cur4, mem4)
            
            spk1_rec.append(spk1)
            spk2_rec.append(spk2)
            spk3_rec.append(spk3)
            spk4_rec.append(spk4)
            mem4_rec.append(mem4)

        return torch.stack(spk4_rec, dim=0), torch.stack(mem4_rec, dim=0), torch.stack(spk1_rec, dim=0), torch.stack(spk2_rec, dim=0), torch.stack(spk3_rec, dim=0)

# Load the network onto CUDA if available
net = Net().to(device)

In [8]:
def print_batch_accuracy(data, targets, train=False):
    x=spikegen.rate(data.view(batch_size,-1),num_steps=num_steps)
    output, _,_,_,_ = net(x)
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer(
    data, targets, epoch,
    counter, iter_counter,
        loss_hist, test_loss_hist, test_data, test_targets):
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print("\n")

## Training the Model

In [9]:
loss = nn.CrossEntropyLoss()

In [10]:
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))

In [11]:
import time
start=time.time()

num_epochs = 1
loss_hist = []
test_loss_hist = []
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        x=spikegen.rate(data.view(batch_size,-1),num_steps=num_steps)
        spk_rec, mem_rec,_,_,_ = net(x)

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            loss_val += loss(mem_rec[step], targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device)
            test_targets = test_targets.to(device)

            # Test set forward pass
            x=spikegen.rate(test_data.view(batch_size,-1),num_steps=num_steps)
            test_spk, test_mem,_,_,_ = net(x)

            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_mem[step], test_targets)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            if counter % 50 == 0:
                train_printer(
                    data, targets, epoch,
                    counter, iter_counter,
                    loss_hist, test_loss_hist,
                    test_data, test_targets)
            counter += 1
            iter_counter +=1

Epoch 0, Iteration 0
Train Set Loss: 57.61
Test Set Loss: 57.45
Train set accuracy for a single minibatch: 10.16%
Test set accuracy for a single minibatch: 14.06%


Epoch 0, Iteration 50
Train Set Loss: 26.19
Test Set Loss: 24.99
Train set accuracy for a single minibatch: 87.50%
Test set accuracy for a single minibatch: 89.84%


Epoch 0, Iteration 100
Train Set Loss: 16.81
Test Set Loss: 11.09
Train set accuracy for a single minibatch: 87.50%
Test set accuracy for a single minibatch: 93.75%


Epoch 0, Iteration 150
Train Set Loss: 12.87
Test Set Loss: 10.95
Train set accuracy for a single minibatch: 92.19%
Test set accuracy for a single minibatch: 95.31%


Epoch 0, Iteration 200
Train Set Loss: 12.19
Test Set Loss: 13.16
Train set accuracy for a single minibatch: 94.53%
Test set accuracy for a single minibatch: 95.31%


Epoch 0, Iteration 250
Train Set Loss: 9.76
Test Set Loss: 12.22
Train set accuracy for a single minibatch: 96.88%
Test set accuracy for a single minibatch: 91.41%


Ep

In [ ]:
# Plot Loss
fig = plt.figure(facecolor="w", figsize=(10, 5))
plt.plot(loss_hist)
plt.plot(test_loss_hist)
plt.title("Loss Curves")
plt.legend(["Train Loss", "Test Loss"])
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

# Load Weights

In [15]:
# # Exporting weights
# import pickle
# with open('weightsMNIST25.pkl', 'wb') as f:
#     pickle.dump(net.state_dict(), f)

In [16]:
# Loading weights
import pickle
with open('weightsMNIST25.pkl', 'rb') as f:
    weights = pickle.load(f)

# Set the weights to the model
net.load_state_dict(weights)

<All keys matched successfully>

## Accuracy

In [17]:
total = 0
correct = 0

# drop_last switched to False to keep all samples
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=False)

with torch.no_grad():
  net.eval()
  for data, targets in test_loader:
    data = data.to(device)
    targets = targets.to(device)

    # forward pass
    x=spikegen.rate(data.view(data.size(0),-1),num_steps=num_steps)
    test_spk, _,_,_,_ = net(x)

    # calculate total accuracy
    _, predicted = test_spk.sum(dim=0).max(1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()

print(f"Total correctly classified test set images: {correct}/{total}")
print(f"Test Set Accuracy: {100 * correct / total:.2f}%")

Total correctly classified test set images: 9459/10000
Test Set Accuracy: 94.59%


## Extracting Weights

In [18]:
def find_params(model):
  weights = []
  biases = []

  for param_tensor in model.state_dict():
      param = model.state_dict()[param_tensor].numpy()
      if 'weight' in param_tensor:
          weights.append(param)
      elif 'bias' in param_tensor:
          biases.append(param)

  return weights, biases

## Creating the List of all Neurons in the Model

In [19]:
def generate_list_of_all_neurons(w):
    global neurons_list
    for i in range(len(w)):
        for j in range(len(w[i])):
            neurons_list.append(f'X{i+1}_{j+1}')

## 1000 random spike using PyTorch/SNN Torch

In [20]:
size=(1,784)

In [21]:
torch.rand(size, dtype=torch.float32)

tensor([[2.9602e-01, 8.9431e-01, 6.1417e-01, 6.9018e-01, 2.0671e-01, 8.1359e-01,
         7.1632e-01, 8.9292e-01, 4.1440e-01, 5.8646e-01, 5.9115e-01, 6.2694e-01,
         1.6264e-02, 8.6148e-01, 5.9721e-01, 7.8307e-01, 1.6982e-01, 5.5911e-01,
         5.0912e-01, 5.0419e-02, 4.7136e-01, 4.2804e-01, 3.5596e-01, 2.8594e-01,
         2.3200e-03, 2.5437e-01, 9.2418e-01, 1.8583e-01, 2.3610e-01, 3.0560e-01,
         6.1876e-01, 3.0583e-02, 3.9316e-02, 4.4815e-01, 6.8450e-01, 3.0788e-01,
         5.3530e-01, 9.2126e-01, 3.1239e-01, 1.8196e-01, 1.7303e-01, 5.4099e-01,
         9.9636e-01, 2.9551e-01, 5.8840e-01, 2.3330e-02, 9.6221e-01, 7.6504e-01,
         7.4739e-01, 1.1211e-01, 7.8578e-01, 4.6199e-01, 2.1695e-01, 5.2928e-02,
         9.8745e-01, 2.2976e-01, 9.0416e-01, 5.7851e-01, 8.0210e-01, 7.8008e-01,
         5.2464e-01, 6.2511e-01, 8.6471e-03, 1.1593e-02, 6.2674e-01, 2.9234e-02,
         1.6276e-02, 2.8533e-01, 6.6739e-01, 2.6661e-01, 8.9427e-02, 9.0428e-01,
         8.7216e-01, 4.0879e

In [36]:
def rand_1000():
    global neurons_list
    size = (1,784)
    for i in range (1000):
#         tensor = torch.randint(0, 2, size, dtype=torch.float32)
        tensor = torch.rand(size, dtype=torch.float32)
        x=spikegen.rate(data.view(data.size(0),-1),num_steps=num_steps)
        spk_4, _,spk_1, spk_2, spk_3 = net(x)
#         res=list_no_spikes(spk_2,spk_1,spk_3,spk_4)
        res=remove_spiked_neurons(spk_4,spk_1,spk_2,spk_3)
        if(i%100==0):
            print("\nNeurons left to spike after ",i+100," random input spike trains: ",len(neurons_list),end=" ")

## Functions to remove neurons that spike from a generated test

In [25]:
def remove_spiked_neurons(spk_4,spk_1,spk_2,spk_3):
    global neurons_list
    col_sum1 = torch.sum(spk_1, dim=0)
    col_sum2 = torch.sum(spk_2, dim=0)
    col_sum3 = torch.sum(spk_3, dim=0)
    col_sum4 = torch.sum(spk_4, dim=0)
    
    spikes_1 = torch.nonzero(col_sum1[0] >= 1).flatten()
    spikes_2 = torch.nonzero(col_sum2[0] >= 1).flatten()
    spikes_3 = torch.nonzero(col_sum3[0] >= 1).flatten()
    spikes_4 = torch.nonzero(col_sum4[0] >= 1).flatten()
    
    slist=[]
    for zp in spikes_1:
        remove=zp+1
        neuron=f'X1_{remove}'
        if neuron not in slist:
            slist.append(neuron)
    for zp in spikes_2:
        remove=zp+1
        neuron=f'X2_{remove}'
        if neuron not in slist:
            slist.append(neuron)
    for zp in spikes_3:
        remove=zp+1
        neuron=f'X3_{remove}'
        if neuron not in slist:
            slist.append(neuron)

    for zp in spikes_4:
        remove=zp+1
        neuron=f'X4_{remove}'
        if neuron not in slist:
            slist.append(neuron)
#     print("Neurons spiking in PyTorch simulation:")
#     print(slist)
    neurons_list = [item for item in neurons_list if item not in slist]
#     return neurons_list
    return slist

## Function to use a generated test in attempt to fire other neurons

In [57]:
def use_generated_test_on_concrete(model):
    inp_tensor = [[[0.0] * 784 for _ in range(num_steps)]]
    decls=model.decls()
    for d in decls:
        input_string=f"{d.name()}"
        if input_string.startswith('X0'):
            first_number = input_string.split('_')[0][1:]  # Extract the number after 'X'
            second_number = input_string.split('_')[1] 
            third_number = input_string.split('_')[2] 
    #             if is_true(model[d]):
            if model[d]==1:
                numeric_value = 1.0
            else:
                numeric_value = 0.0
    #         print(third_number,second_number)
            inp_tensor[0][int(third_number)-1][int(second_number)-1]=numeric_value
    tensor=torch.tensor(inp_tensor)
    ten=tensor.view(25,1,784)
    print("Input to PyTorch:")
    print(ten)
    spk_4, _,spk_1,spk_2, spk_3 = net(ten)
    s_list=remove_spiked_neurons(spk_4,spk_1,spk_2,spk_3)
    return s_list

# Function for test generation using cone of influence

In [43]:
def generate_test(w,b,layer,no,time_steps):
    
    equations=[]
    declare=[]
    inputs=w[0].shape[1]
    solver=Solver()
    solver.reset()
    solver.set(timeout=240000)

      # Declarations
    for time in range(1,time_steps+1):
      for num in range(1,inputs+1):
        decl=f"X0_{num}_{time}=Real('X0_{num}_{time}')"

        exec(decl)
        declare.append(decl)
        exe=f"solver.add(Or(X0_{num}_{time}==0,X0_{num}_{time}==1))"
        exec(exe)

    for i in range(1,layer):
      for j in range(1,len(w[i-1])+1):
        decl=f"P{i}_{j}_0=Real('P{i}_{j}_0')"
        exec(decl)
        declare.append(decl)

    decl=f"P{layer}_{no}_0=Real('P{layer}_{no}_0')"
    exec(decl)
    declare.append(decl)

    for time in range(1,time_steps+1):
      for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
          decl=f"X{i}_{j}_{time}=Real('X{i}_{j}_{time}')"
          exec(decl)
          declare.append(decl)
          decl=f"P{i}_{j}_{time}=Real('P{i}_{j}_{time}')"
          exec(decl)
          declare.append(decl)
          decl=f"S{i}_{j}_{time}=Real('S{i}_{j}_{time}')"
          exec(decl)
          declare.append(decl)

    for time in range(1,time_steps+1):
      decl=f"X{layer}_{no}_{time}=Real('X{layer}_{no}_{time}')"
      exec(decl)
      declare.append(decl)
      decl=f"P{layer}_{no}_{time}=Real('P{layer}_{no}_{time}')"
      exec(decl)
      declare.append(decl)
      decl=f"S{layer}_{no}_{time}=Real('S{layer}_{no}_{time}')"
      exec(decl)
      declare.append(decl)

    # Encodings
    for i in range(1,layer):
      for j in range(1,len(w[i-1])+1):
        equations.append(f'P{i}_{j}_0== 0')
        exe=f"solver.add(P{i}_{j}_0== 0)"
        exec(exe)

    equations.append(f'P{layer}_{no}_0== 0')
    exe=f"solver.add(P{layer}_{no}_0== 0)"
    exec(exe)

    lamb=Real('lamb')
    thresh=Real('thresh')
    solver.add(lamb==1)
    solver.add(thresh==1)

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f'Implies((S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1}) >= thresh,X{i}_{j}_{time}==1)')
                exe=f"solver.add(Implies((S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1}) >= thresh,X{i}_{j}_{time}==1))"
                exec(exe)
                equations.append(f'Implies((S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1}) < thresh,X{i}_{j}_{time}==0)')
                exe=f"solver.add(Implies((S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1}) < thresh,X{i}_{j}_{time}==0))"
                exec(exe)
                equations.append(f'Implies((S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1}) >= thresh,P{i}_{j}_{time}==(S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1})-thresh ))')
                exe=f"solver.add(Implies((S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1}) >= thresh,P{i}_{j}_{time}==(S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1})-thresh ))"
                exec(exe)
                equations.append(f'Implies((S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1}) < thresh,P{i}_{j}_{time}==(S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1})')
                exe=f"solver.add(Implies((S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1}) < thresh,P{i}_{j}_{time}==(S{i}_{j}_{time} + lamb * P{i}_{j}_{time-1})))"
                exec(exe)
                equation = f'S{i}_{j}_{time} == ('
                for k in range(len(w[i-1][0])):            
                    if(k!=0):
                        equation += f' + '
                    equation+=f'({w[i-1][j-1][k]:.8f} * X{i-1}_{k+1}_{time})'
                exe=f"solver.add({equation}))"
                equations.append(equation)
                exec(exe)

    for time in range(1,time_steps+1):
        equations.append(f'Implies((S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}) >= thresh,X{layer}_{no}_{time}==1)')
        exe=f"solver.add(Implies((S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}) >= thresh,X{layer}_{no}_{time}==1))"
        exec(exe)
        equations.append(f'Implies((S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}) < thresh,X{layer}_{no}_{time}==0)')
        exe=f"solver.add(Implies((S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}) < thresh,X{layer}_{no}_{time}==0))"
        exec(exe)
        equations.append(f'Implies((S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}) >= thresh,P{layer}_{no}_{time}==(S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1})-thresh )')
        exe=f"solver.add(Implies((S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}) >= thresh,P{layer}_{no}_{time}==(S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1})-thresh ))"
        exec(exe)
        equations.append(f'Implies((S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}) < thresh,P{layer}_{no}_{time}==S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1})')
        exe=f"solver.add(Implies((S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}) < thresh,P{layer}_{no}_{time}==S{layer}_{no}_{time} + lamb * P{layer}_{no}_{time-1}))"
        exec(exe)
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):            
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]:.8f}) * X{layer-1}_{k+1}_{time})'
        exe=f"solver.add({equation}))"
        equations.append(equation)
        exec(exe)
    cond=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            cond+=f','
        cond+=f'X{layer}_{no}_{time}==1'
    exe=f'solver.add(Or({cond}))'
    equations.append(exe)
    exec(exe)
    res=solver.check()
    if res==unsat:
        print('Will never spike')
        return 0
    elif res==sat:
        print("Has spiked")
        model=solver.model()
        return model
    else:
        print("Unknown from solver")
        return 1

In [44]:
def neurons_spiking_in_smt(res):
    decls=res.decls()
    decl_list=[]
    for d in decls:
        if res[d]==1 and str(d).startswith('X'):
            decl_list.append(str(d))
    decl_list.sort()
    grouped_dict = {}

# Iterate through the list and group the strings by their first two characters
    for string in decl_list:
        key = string[:2]  # Get the first two characters as the key
        if key not in grouped_dict:
            grouped_dict[key] = []
        grouped_dict[key].append(string)
    multi_dimensional_list = list(grouped_dict.values())
    for l in multi_dimensional_list:
        print(l)

In [45]:
def neurons_spiking_in_pytorch(tensor):
    spk_4, _,spk_1,spk_2, spk_3=net(tensor)
    col_sum1 = torch.sum(spk_1, dim=0)
    col_sum2 = torch.sum(spk_2, dim=0)
    col_sum3 = torch.sum(spk_3, dim=0)
    col_sum4 = torch.sum(spk_4, dim=0)
    
    spikes_1 = torch.nonzero(col_sum1[0] >= 1).flatten()
    spikes_2 = torch.nonzero(col_sum2[0] >= 1).flatten()
    spikes_3 = torch.nonzero(col_sum3[0] >= 1).flatten()
    spikes_4 = torch.nonzero(col_sum4[0] >= 1).flatten()
    
    
    slist=[]
    for zp in spikes_1:
        remove=zp+1
        neuron=f'X1_{remove}'
        if neuron not in slist:
            slist.append(neuron)
    for zp in spikes_2:
        remove=zp+1
        neuron=f'X2_{remove}'
        if neuron not in slist:
            slist.append(neuron)
    for zp in spikes_3:
        remove=zp+1
        neuron=f'X3_{remove}'
        if neuron not in slist:
            slist.append(neuron)

    for zp in spikes_4:
        remove=zp+1
        neuron=f'X4_{remove}'
        if neuron not in slist:
            slist.append(neuron)
    # slist.sort()
    print("Neurons spiking in PyTorch simulation:")
    print(slist)

# Abstraction

In [27]:
def create_list_of_lists(num_lists):    
    list_of_lists = [[] for _ in range(num_lists)]
    return list_of_lists

In [28]:
def abstraction(w,b,out_layer,n):
#     print("Neurons in the first hidden layer: ",len(w[0]))
#     print("Neurons in the second hidden layer: ",len(w[1]))
#     print("Neurons in the third hidden layer: ",len(w[2]))
#     print("Neurons in the fourth hidden layer: ",len(w[3]))
    w=w[:out_layer]
    b=b[:out_layer]
    
    last_array = np.array(w[-1][n-1])
    last_array = np.expand_dims(last_array, axis=0)
    w=w[:-1]
    w.append(last_array)
    
    last_array = np.array(b[-1][n-1])
    last_array = np.expand_dims(last_array, axis=0)

    b=b[:-1]
    b.append(last_array)
    
    for i in range(len(w)-1):
        print("Layer",i+1,":",len(w[i]))
    pos_neg = [['+' for _ in row] for row in b[:-1]]
    wc=w[:]
    for i in range(1,len(wc)):
        for j in range(len(wc[i-1])):
            flag=0
            no_pos_flag=1
            add_weights=[]
            for k in range(len(w[i])):
                if(wc[i][k][j]<0):
                    flag=1
                    add_weights.append(wc[i][k][j])
                else:
                    add_weights.append(0)
                if(wc[i][k][j]>0):
                    no_pos_flag=0
            if(no_pos_flag==1):
                pos_neg[i-1][j]='-'
                continue
            if(flag==1):
#                 for k in range(len(w[i])):
#                     if(wc[i][k][j]<0):
#                         wc[i][k][j]=0
                wc[i-1]=np.concatenate((wc[i-1],wc[i-1][j].reshape(1, -1)), axis=0)
                add_weights=np.array(add_weights)
                wc[i]=np.concatenate((wc[i],add_weights.reshape(-1, 1)), axis=1)
                pos_neg[i-1].append('-')
                for k in range(len(w[i])):
                    if(wc[i][k][j]<0):
                        wc[i][k][j]=0
#     print("Neurons in the first hidden layer: ",len(wc[0]))
#     print("Neurons in the second hidden layer: ",len(wc[1]))
#     print("Neurons in the third hidden layer: ",len(wc[2]))
#     print("Neurons in the third hidden layer: ",len(wc[3]))

    for i in range(len(wc)-1):
        print("Layer",i+1,":",len(wc[i]))
    inc_dec=[]
    for sub_array in wc:
        inc_dec.append([''] * len(sub_array))
    inc_dec[-1] = ['I'] * len(inc_dec[-1])
    
    wc2=wc[:]
    for i in range(len(wc)-1,0,-1):
        for j in range(len(wc[i-1])):
    #         print(i,end=" ")
            dec_flag=0
            inc_flag=0  # Indicates none
            iterate=len(wc2[i])
    #         print(iterate, end="  ")
            for k in range(iterate):
                if(pos_neg[i-1][j]=='+' and inc_dec[i][k]=='I'): # Inc----
                    inc_flag=1
    #                 print("Case 1")
                if(pos_neg[i-1][j]=='-' and inc_dec[i][k]=='D'): # Inc
                    inc_flag=1
    #                 print("Case 2")
                if(pos_neg[i-1][j]=='+' and inc_dec[i][k]=='D'): # Dec
                    dec_flag=1
    #                 print("Case 3")
                if(pos_neg[i-1][j]=='-' and inc_dec[i][k]=='I'): # Dec----
                    dec_flag=1
    #                 print("Case 4")

            if dec_flag==1 and inc_flag==0:
                inc_dec[i-1][j]='D'
            elif dec_flag==0 and inc_flag==1:
                inc_dec[i-1][j]='I'

            if dec_flag==1 and inc_flag==1:
    #             print("Special case")
                weights=[]
                inc_dec[i-1][j]='I'
                inc_dec[i-1].append('D')
                if(pos_neg[i-1][j]=='+'):
                    pos_neg[i-1].append('+')
                else:
                    pos_neg[i-1].append('-')

                for k in range(iterate):
    #             for k in range(len(wc[i])):
                    if(pos_neg[i-1][j]=='+' and inc_dec[i][k]=='I'): # Inc
                        weights.append(0)
                    elif(pos_neg[i-1][j]=='-' and inc_dec[i][k]=='D'): # Inc
                        weights.append(0)
                    elif(pos_neg[i-1][j]=='-' and inc_dec[i][k]=='I'): # Dec
                        weights.append(wc2[i][k][j])
    #                     wc[i][k][j]=0
                        wc2[i][k][j]=0
                    elif(pos_neg[i-1][j]=='+' and inc_dec[i][k]=='D'): # Dec
                        weights.append(wc2[i][k][j])
    #                     wc[i][k][j]=0
                        wc2[i][k][j]=0
                    elif(wc2[i][k][j]==0.0):
                        weights.append(0)                
                wc2[i-1]=np.concatenate((wc2[i-1],wc2[i-1][j].reshape(1, -1)), axis=0)
                weights=np.array(weights)
                wc2[i]=np.concatenate((wc2[i],weights.reshape(-1, 1)), axis=1)
#     print("Neurons in the first hidden layer: ",len(wc2[0]))
#     print("Neurons in the second hidden layer: ",len(wc2[1]))
#     print("Neurons in the third hidden layer: ",len(wc2[2]))
#     print("Neurons in the third hidden layer: ",len(wc[3]))
    for i in range(len(wc2)-1):
        print("Layer",i+1,":",len(wc2[i]))
    layers=len(wc2)-1
    po_in = create_list_of_lists(layers)
    po_de = create_list_of_lists(layers)
    ne_in = create_list_of_lists(layers)
    ne_de = create_list_of_lists(layers)
    for i in range(len(wc2)-1):
        for j in range(len(wc2[i])):
            if pos_neg[i][j]=='+' and inc_dec[i][j]=='I':
                po_in[i].append(j)
            if pos_neg[i][j]=='+' and inc_dec[i][j]=='D':
                po_de[i].append(j)
            if pos_neg[i][j]=='-' and inc_dec[i][j]=='I':
                ne_in[i].append(j)
            if pos_neg[i][j]=='-' and inc_dec[i][j]=='D':
                ne_de[i].append(j)
    columnwise_max_po_in=[] 
    columnwise_max_ne_de=[] 
    columnwise_max_ne_in=[] 
    columnwise_max_po_de=[]

    columnwise_sum_po_in=[] 
    columnwise_sum_ne_de=[] 
    columnwise_sum_ne_in=[] 
    columnwise_sum_po_de=[]

    for i in range(len(wc2)-1):    
        if np.any(po_in[i]):
            po_in_max = wc2[i][po_in[i]]
            columnwise_max_po_in = np.max(po_in_max, axis=0)
            po_in_sum = wc2[i+1][:,po_in[i]]
            columnwise_sum_po_in = np.sum(po_in_sum, axis=1)

        if np.any(ne_de[i]):
            ne_de_max = wc2[i][ne_de[i]]
            columnwise_max_ne_de = np.max(ne_de_max, axis=0)
            ne_de_sum = wc2[i+1][:,ne_de[i]]
            columnwise_sum_ne_de = np.sum(ne_de_sum, axis=1)

        if np.any(ne_in[i]):
            ne_in_max = wc2[i][ne_in[i]]
            columnwise_max_ne_in = np.max(ne_in_max, axis=0)
            ne_in_sum = wc2[i+1][:,ne_in[i]]
            columnwise_sum_ne_in = np.sum(ne_in_sum, axis=1)

        if np.any(po_de[i]):
            po_de_max = wc2[i][po_de[i]]
            columnwise_max_po_de = np.max(po_de_max, axis=0)
            po_de_sum = wc2[i+1][:,po_de[i]]
            columnwise_sum_po_de = np.sum(po_de_sum, axis=1)

        tensors_to_stack_inp = [tensor for tensor in [columnwise_max_po_in, columnwise_max_ne_de, columnwise_max_ne_in, columnwise_max_po_de] if np.any(tensor)]
        tensors_to_stack_out = [tensor for tensor in [columnwise_sum_po_in, columnwise_sum_ne_de, columnwise_sum_ne_in, columnwise_sum_po_de] if np.any(tensor)]
        tensors_to_stack_out=np.array(tensors_to_stack_out).T

        wc2[i]=tensors_to_stack_inp
        wc2[i+1]=tensors_to_stack_out

        columnwise_max_po_in=[] 
        columnwise_max_ne_de=[] 
        columnwise_max_ne_in=[] 
        columnwise_max_po_de=[]

        columnwise_sum_po_in=[] 
        columnwise_sum_ne_de=[] 
        columnwise_sum_ne_in=[] 
        columnwise_sum_po_de=[]
        tensors_to_stack_inp=[]
        tensors_to_stack_out=[]
    
    for i in range(len(wc2)-1):
        print("Layer",i+1,":",len(wc2[i]))
    return wc2

In [66]:
neurons_list=[]

In [67]:
w_net,b_net=find_params(net)

In [68]:
generate_list_of_all_neurons(w_net)

In [69]:
neurons_list

['X1_1',
 'X1_2',
 'X1_3',
 'X1_4',
 'X1_5',
 'X1_6',
 'X1_7',
 'X1_8',
 'X1_9',
 'X1_10',
 'X1_11',
 'X1_12',
 'X1_13',
 'X1_14',
 'X1_15',
 'X1_16',
 'X1_17',
 'X1_18',
 'X1_19',
 'X1_20',
 'X1_21',
 'X1_22',
 'X1_23',
 'X1_24',
 'X1_25',
 'X1_26',
 'X1_27',
 'X1_28',
 'X1_29',
 'X1_30',
 'X1_31',
 'X1_32',
 'X1_33',
 'X1_34',
 'X1_35',
 'X1_36',
 'X1_37',
 'X1_38',
 'X1_39',
 'X1_40',
 'X1_41',
 'X1_42',
 'X1_43',
 'X1_44',
 'X1_45',
 'X1_46',
 'X1_47',
 'X1_48',
 'X1_49',
 'X1_50',
 'X1_51',
 'X1_52',
 'X1_53',
 'X1_54',
 'X1_55',
 'X1_56',
 'X1_57',
 'X1_58',
 'X1_59',
 'X1_60',
 'X1_61',
 'X1_62',
 'X1_63',
 'X1_64',
 'X1_65',
 'X1_66',
 'X1_67',
 'X1_68',
 'X1_69',
 'X1_70',
 'X1_71',
 'X1_72',
 'X1_73',
 'X1_74',
 'X1_75',
 'X1_76',
 'X1_77',
 'X1_78',
 'X1_79',
 'X1_80',
 'X1_81',
 'X1_82',
 'X1_83',
 'X1_84',
 'X1_85',
 'X1_86',
 'X1_87',
 'X1_88',
 'X1_89',
 'X1_90',
 'X1_91',
 'X1_92',
 'X1_93',
 'X1_94',
 'X1_95',
 'X1_96',
 'X1_97',
 'X1_98',
 'X1_99',
 'X1_100',
 'X1_101

In [70]:
rand_1000()


Neurons left to spike after  100  random input spike trains:  102 
Neurons left to spike after  200  random input spike trains:  89 
Neurons left to spike after  300  random input spike trains:  89 
Neurons left to spike after  400  random input spike trains:  88 
Neurons left to spike after  500  random input spike trains:  87 
Neurons left to spike after  600  random input spike trains:  87 
Neurons left to spike after  700  random input spike trains:  87 
Neurons left to spike after  800  random input spike trains:  87 
Neurons left to spike after  900  random input spike trains:  87 
Neurons left to spike after  1000  random input spike trains:  87 

In [71]:
print("Candidate dead neurons: ",neurons_list) # Candidate dead neurons

Candidate dead neurons:  ['X1_10', 'X1_11', 'X1_22', 'X1_23', 'X1_28', 'X1_30', 'X1_31', 'X1_32', 'X1_36', 'X1_38', 'X1_42', 'X1_50', 'X1_52', 'X1_55', 'X1_56', 'X1_58', 'X1_59', 'X1_60', 'X1_63', 'X1_64', 'X1_71', 'X1_72', 'X1_77', 'X1_80', 'X1_84', 'X1_87', 'X1_90', 'X1_94', 'X1_99', 'X1_101', 'X1_104', 'X1_107', 'X1_109', 'X1_111', 'X1_112', 'X1_121', 'X1_125', 'X1_126', 'X1_127', 'X1_129', 'X1_132', 'X1_134', 'X1_135', 'X1_136', 'X1_137', 'X1_139', 'X1_140', 'X1_145', 'X1_152', 'X1_153', 'X1_160', 'X1_163', 'X1_166', 'X1_171', 'X1_175', 'X1_184', 'X1_185', 'X1_187', 'X1_191', 'X1_192', 'X1_194', 'X1_199', 'X1_200', 'X2_4', 'X2_6', 'X2_17', 'X2_24', 'X2_29', 'X2_30', 'X2_38', 'X2_47', 'X2_49', 'X2_60', 'X2_63', 'X2_86', 'X2_93', 'X2_94', 'X2_100', 'X3_10', 'X3_13', 'X3_30', 'X3_35', 'X4_3', 'X4_4', 'X4_5', 'X4_8', 'X4_10']


In [40]:
# neurons_list=['X1_2', 'X1_16', 'X1_21', 'X1_26', 'X1_28', 'X1_31', 'X1_39', 'X1_44', 'X1_55', 'X1_68', 'X1_75', 'X1_80', 'X1_93', 'X1_100', 'X1_102', 'X1_105', 'X1_106', 'X1_112', 'X1_115', 'X1_117', 'X1_120', 'X1_124', 'X1_126', 'X1_131', 'X1_132', 'X1_133', 'X1_134', 'X1_135', 'X1_146', 'X1_151', 'X1_152', 'X1_156', 'X1_165', 'X1_166', 'X1_167', 'X1_178', 'X1_180', 'X1_184', 'X1_191', 'X1_192', 'X1_195', 'X1_196', 'X2_81', 'X2_88', 'X2_100', 'X3_15', 'X3_38', 'X3_41', 'X3_44', 'X4_5', 'X4_10']

In [39]:
net

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=False)
  (lif1): Leaky()
  (fc2): Linear(in_features=200, out_features=100, bias=False)
  (lif2): Leaky()
  (fc3): Linear(in_features=100, out_features=50, bias=False)
  (lif3): Leaky()
  (fc4): Linear(in_features=50, out_features=10, bias=False)
  (lif4): Leaky()
)

In [72]:
import time
unknown_list=[]
dead_list=[]
unknown_ts=[]
time_steps=5
spiked_list=[]
spiked_ts=[]
# for i in range(len(neurons_list)):
#     item=neurons_list[i]
index = 0
while index < len(neurons_list):
    item=neurons_list[0]
    layer_no = int(item.split('_')[0][1:])
    neuron_no = int(item.split('_')[1])
    for j in range(1,time_steps+1):
        print(item," upto time step ",j," : ",end="")
        start_time = time.time()
        res = generate_test(w_net,b_net,layer_no,neuron_no,j)
        end_time = time.time()
        print("Time for test generation: ",end_time-start_time)
        if res==0: # dead
            if j>=time_steps:
                dead_list.append(item)
                if item in neurons_list:
                    neurons_list.remove(item)
            continue
#             if item in neurons_list:
#                 neurons_list.remove(item)
        elif res==1: # unknown
            unknown_list.append(item) 
            unknown_ts.append(j)
            if item in neurons_list:
                neurons_list.remove(item)
            break
#             if item in neurons_list:
#                 neurons_list.remove(item)
#             unknown=unknown+1
#             if(unknown>1):
#                 break
        else:
            spiked_ts.append(j)
            spiked_list.append(item)
            spike_py=use_generated_test_on_concrete(res)
            print('Neurons spiking in PyTorch simulation:')
            print(spike_py)
            
#             neurons_spiking_in_pytorch(spike_py)
            print('Neurons spiking in SMT simulation:')
            neurons_spiking_in_smt(res)
            break
#             use_generated_test(res)
    print(len(neurons_list),' neurons remaining...',end='\n\n')

X1_10  upto time step  1  : Has spiked
Time for test generation:  17.98344397544861
Input to PyTorch:
tensor([[[1., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_5', 'X1_9', 'X1_10', 'X1_26', 'X1_62', 'X1_82', 'X1_105', 'X1_177', 'X1_187']
Neurons spiking in SMT simulation:
['X0_10_1', 'X0_113_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_126_1', 'X0_128_1', 'X0_133_1', 'X0_136_1', 'X0_143_1', 'X0_147_1', 'X0_14_1', 'X0_154_1', 'X0_155_1', 'X0_157_1', 'X0_165_1', 'X0_16_1', 'X0_171_1', 'X0_175_1', 'X0_178_1', 'X0_17_1', 'X0_183_1', 'X0_187_1', 'X0_18_1', 'X0_197_1', 'X0_19_1', 'X0_1_1', 'X0_205_1', 'X0_214_1', 'X0_219_1', 'X0_21_1', 'X0_227_1', 'X0_229_1', 'X0_231_1', 'X0_233_1', 'X0_235_1', 'X0_23_1', 'X0_240_1', 'X0_242_1',

X1_30  upto time step  1  : Has spiked
Time for test generation:  17.288284301757812
Input to PyTorch:
tensor([[[1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_9', 'X1_30', 'X1_174', 'X1_177']
Neurons spiking in SMT simulation:
['X0_103_1', 'X0_10_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_123_1', 'X0_127_1', 'X0_128_1', 'X0_130_1', 'X0_133_1', 'X0_136_1', 'X0_139_1', 'X0_143_1', 'X0_147_1', 'X0_14_1', 'X0_156_1', 'X0_157_1', 'X0_165_1', 'X0_16_1', 'X0_171_1', 'X0_178_1', 'X0_183_1', 'X0_18_1', 'X0_197_1', 'X0_19_1', 'X0_1_1', 'X0_205_1', 'X0_219_1', 'X0_227_1', 'X0_233_1', 'X0_23_1', 'X0_242_1', 'X0_25_1', 'X0_26_1', 'X0_27_1', 'X0_289_1', 'X0_28_1', 'X0_294_1', 'X0_307_1', 'X0_309_1', 'X0_30_1', 'X0_31_1', 'X0_33_1', 

X1_42  upto time step  1  : Has spiked
Time for test generation:  17.25025177001953
Input to PyTorch:
tensor([[[1., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_5', 'X1_9', 'X1_42', 'X1_62', 'X1_177']
Neurons spiking in SMT simulation:
['X0_104_1', 'X0_10_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_128_1', 'X0_133_1', 'X0_140_1', 'X0_143_1', 'X0_147_1', 'X0_14_1', 'X0_155_1', 'X0_156_1', 'X0_157_1', 'X0_165_1', 'X0_16_1', 'X0_171_1', 'X0_177_1', 'X0_178_1', 'X0_183_1', 'X0_18_1', 'X0_197_1', 'X0_19_1', 'X0_1_1', 'X0_205_1', 'X0_206_1', 'X0_219_1', 'X0_21_1', 'X0_227_1', 'X0_234_1', 'X0_235_1', 'X0_23_1', 'X0_242_1', 'X0_246_1', 'X0_253_1', 'X0_254_1', 'X0_255_1', 'X0_25_1', 'X0_260_1', 'X0_263_1', 'X0_267_1', 'X0_26_1', 'X

X1_58  upto time step  1  : Has spiked
Time for test generation:  17.250762224197388
Input to PyTorch:
tensor([[[1., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_58', 'X1_193']
Neurons spiking in SMT simulation:
['X0_103_1', 'X0_10_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_127_1', 'X0_128_1', 'X0_133_1', 'X0_136_1', 'X0_143_1', 'X0_147_1', 'X0_14_1', 'X0_156_1', 'X0_157_1', 'X0_165_1', 'X0_16_1', 'X0_171_1', 'X0_178_1', 'X0_183_1', 'X0_18_1', 'X0_197_1', 'X0_19_1', 'X0_1_1', 'X0_205_1', 'X0_219_1', 'X0_21_1', 'X0_227_1', 'X0_232_1', 'X0_233_1', 'X0_23_1', 'X0_242_1', 'X0_243_1', 'X0_255_1', 'X0_256_1', 'X0_25_1', 'X0_262_1', 'X0_263_1', 'X0_267_1', 'X0_26_1', 'X0_289_1', 'X0_28_1', 'X0_294_1', 'X0_307_1', 'X0_309_1', 'X0

X1_71  upto time step  1  : Has spiked
Time for test generation:  17.274181365966797
Input to PyTorch:
tensor([[[1., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_71', 'X1_177']
Neurons spiking in SMT simulation:
['X0_103_1', 'X0_10_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_128_1', 'X0_12_1', 'X0_133_1', 'X0_136_1', 'X0_137_1', 'X0_140_1', 'X0_143_1', 'X0_147_1', 'X0_14_1', 'X0_150_1', 'X0_156_1', 'X0_157_1', 'X0_165_1', 'X0_166_1', 'X0_167_1', 'X0_16_1', 'X0_171_1', 'X0_178_1', 'X0_183_1', 'X0_18_1', 'X0_197_1', 'X0_19_1', 'X0_1_1', 'X0_205_1', 'X0_219_1', 'X0_227_1', 'X0_23_1', 'X0_242_1', 'X0_249_1', 'X0_252_1', 'X0_25_1', 'X0_262_1', 'X0_264_1', 'X0_265_1', 'X0_267_1', 'X0_26_1', 'X0_27_1', 'X0_280_1', 'X0_289_1', 'X0

...

X1_135  upto time step  1  : Has spiked
Time for test generation:  17.42426347732544
Input to PyTorch:
tensor([[[0., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_5', 'X1_9', 'X1_26', 'X1_29', 'X1_62', 'X1_101', 'X1_114', 'X1_135', 'X1_137', 'X1_153', 'X1_174', 'X1_177', 'X1_187', 'X1_193']
Neurons spiking in SMT simulation:
['X0_104_1', 'X0_10_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_128_1', 'X0_133_1', 'X0_143_1', 'X0_147_1', 'X0_150_1', 'X0_156_1', 'X0_157_1', 'X0_165_1', 'X0_16_1', 'X0_171_1', 'X0_178_1', 'X0_17_1', 'X0_183_1', 'X0_18_1', 'X0_197_1', 'X0_19_1', 'X0_205_1', 'X0_219_1', 'X0_21_1', 'X0_227_1', 'X0_228_1', 'X0_231_1', 'X0_233_1', 'X0_234_1', 'X0_235_1', 'X0_23_1', 'X0_241_1', 'X0_242_1', 'X0_243_1', 'X0_

X1_152  upto time step  1  : Has spiked
Time for test generation:  17.357351064682007
Input to PyTorch:
tensor([[[1., 0., 1.,  ..., 1., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_5', 'X1_9', 'X1_62', 'X1_84', 'X1_97', 'X1_137', 'X1_152', 'X1_177', 'X1_193']
Neurons spiking in SMT simulation:
['X0_103_1', 'X0_113_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_127_1', 'X0_128_1', 'X0_133_1', 'X0_134_1', 'X0_135_1', 'X0_136_1', 'X0_143_1', 'X0_147_1', 'X0_14_1', 'X0_150_1', 'X0_156_1', 'X0_157_1', 'X0_165_1', 'X0_16_1', 'X0_171_1', 'X0_178_1', 'X0_183_1', 'X0_18_1', 'X0_194_1', 'X0_197_1', 'X0_19_1', 'X0_1_1', 'X0_205_1', 'X0_206_1', 'X0_219_1', 'X0_21_1', 'X0_227_1', 'X0_233_1', 'X0_23_1', 'X0_242_1', 'X0_25_1', 'X0_261_1', 'X0_262_1', 'X0_265

X1_175  upto time step  1  : Has spiked
Time for test generation:  17.958552837371826
Input to PyTorch:
tensor([[[1., 1., 1.,  ..., 1., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_5', 'X1_9', 'X1_26', 'X1_54', 'X1_62', 'X1_106', 'X1_108', 'X1_138', 'X1_151', 'X1_174', 'X1_175', 'X1_177']
Neurons spiking in SMT simulation:
['X0_103_1', 'X0_10_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_128_1', 'X0_12_1', 'X0_132_1', 'X0_133_1', 'X0_136_1', 'X0_139_1', 'X0_143_1', 'X0_147_1', 'X0_14_1', 'X0_156_1', 'X0_157_1', 'X0_164_1', 'X0_165_1', 'X0_16_1', 'X0_171_1', 'X0_183_1', 'X0_18_1', 'X0_197_1', 'X0_198_1', 'X0_19_1', 'X0_1_1', 'X0_205_1', 'X0_208_1', 'X0_212_1', 'X0_219_1', 'X0_21_1', 'X0_220_1', 'X0_223_1', 'X0_227_1', 'X0_23_1', 'X0_242_1', 'X

X1_194  upto time step  1  : Has spiked
Time for test generation:  17.544352293014526
Input to PyTorch:
tensor([[[1., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
Neurons spiking in PyTorch simulation:
['X1_4', 'X1_5', 'X1_9', 'X1_26', 'X1_62', 'X1_137', 'X1_151', 'X1_177', 'X1_193', 'X1_194', 'X1_196']
Neurons spiking in SMT simulation:
['X0_103_1', 'X0_10_1', 'X0_114_1', 'X0_118_1', 'X0_11_1', 'X0_121_1', 'X0_128_1', 'X0_133_1', 'X0_136_1', 'X0_139_1', 'X0_143_1', 'X0_147_1', 'X0_14_1', 'X0_151_1', 'X0_156_1', 'X0_157_1', 'X0_165_1', 'X0_16_1', 'X0_171_1', 'X0_178_1', 'X0_183_1', 'X0_18_1', 'X0_197_1', 'X0_19_1', 'X0_1_1', 'X0_205_1', 'X0_207_1', 'X0_219_1', 'X0_21_1', 'X0_227_1', 'X0_233_1', 'X0_235_1', 'X0_238_1', 'X0_23_1', 'X0_242_1', 'X0_249_1', 'X0_25_1', 'X0_261_1', '

KeyboardInterrupt: 

In [74]:
dead_list

[]

In [73]:
unknown_list

['X2_4']

In [58]:
spike_py=use_generated_test_on_concrete(res)

Input to PyTorch:
tensor([[[1., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])


In [60]:
len(spike_py)

9

In [50]:
neurons_spiking_in_pytorch(spike_py)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not str

In [64]:
model = res
inp_tensor = [[[0.0] * 784 for _ in range(num_steps)]]
decls=model.decls()
for d in decls:
    input_string=f"{d.name()}"
    if input_string.startswith('X0'):
        first_number = input_string.split('_')[0][1:]  # Extract the number after 'X'
        second_number = input_string.split('_')[1] 
        third_number = input_string.split('_')[2] 
    #             if is_true(model[d]):
        if model[d]==1:
            numeric_value = 1.0
        else:
            numeric_value = 0.0
    #         print(third_number,second_number)
        inp_tensor[0][int(third_number)-1][int(second_number)-1]=numeric_value
tensor=torch.tensor(inp_tensor)
ten=tensor.view(25,1,784)
print("Input to PyTorch:")
print(ten)
spk_4, _,spk_1,spk_2, spk_3 = net(ten)
s_list=remove_spiked_neurons(spk_4,spk_1,spk_2,spk_3)
print(s_list)

Input to PyTorch:
tensor([[[1., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])
['X1_5', 'X1_9', 'X1_11', 'X1_26', 'X1_82', 'X1_108', 'X1_137', 'X1_151', 'X1_153']


In [ ]:
import time
unknown_list=[]
unknown_ts=[]

spiked_list=[]
spiked_ts=[]
for i in range(len(neurons_list)):
    item=neurons_list[i]
    layer_no = int(item.split('_')[0][1:])
    neuron_no = int(item.split('_')[1])
    for j in range(1,time_steps):
        print(item," : ",end="")
        start_time = time.time()
        res = generate_test(w_net,b_net,layer_no,neuron_no,j)
        end_time = time.time()
        print("Time for test generation: ",end_time-start_time)
        if res==0: # dead
#             dead_list.append(item)
            continue
#             if item in neurons_list:
#                 neurons_list.remove(item)
        elif res==1:
            unknown_list.append(item) 
            unknown_ts.append(j)
            break
#             if item in neurons_list:
#                 neurons_list.remove(item)
#             unknown=unknown+1
#             if(unknown>1):
#                 break
        else:
            spiked_ts.append(j)
            spiked_list.append(item)
            use_generated_test(res)

            break
    print(len(neurons_list),' neurons remaining...',end='\n\n')
        
        
    

X1_2  : Has spiked
X1_2 X1_156 X3_44 48  neurons remaining...

X1_21  : Has spiked
X1_21 X1_126 X1_146 X1_167 X3_15 43  neurons remaining...

X1_28  : Has spiked
X1_28 X1_131 X1_135 X1_165 X1_178 X1_192 X1_195 X1_196 X2_100 X3_38 33  neurons remaining...

X1_39  : Has spiked
X1_26 X1_39 X1_117 X1_184 29  neurons remaining...

X1_68  : Has spiked
X1_68 X1_100 X3_41 X4_10 25  neurons remaining...

X1_80  : Has spiked
X1_80 24  neurons remaining...

X1_102  : Has spiked
X1_102 X1_132 X1_180 21  neurons remaining...

X1_106  : Has spiked
X1_106 X1_151 X1_152 18  neurons remaining...

X1_115  : Has spiked
X1_115 17  neurons remaining...

X1_124  : Has spiked
X1_44 X1_124 X1_166 14  neurons remaining...

X1_191  : Has spiked
X1_191 X2_81 12  neurons remaining...

X4_5  : 

In [ ]:
spiked_list

In [ ]:
unknown_list

# Test generation for 1 timestep

In [33]:
import time
remaining_list=[]
time_steps=1
unknown=0
i=0
print("Generating test cases for the candidate dead neurons:",end="\n\n")
start=time.time()
index = 0
while index < len(neurons_list):
    item=neurons_list[0]
    layer_no = int(item.split('_')[0][1:])
    neuron_no = int(item.split('_')[1])
    
    print(item," : ",end="")
    start_time = time.time()
    res = generate_test(w_net,b_net,layer_no,neuron_no,time_steps)
    end_time = time.time()
    print("Time for test generation: ",end_time-start_time)
    if res!=1:
        use_generated_test(res)
    else:
        remaining_list.append(item)
        unknown=unknown+1
#         if(unknown>1):
#             break


#     if item in neurons_list:
#         neurons_list.remove(item)
    print(len(neurons_list),' neurons remaining...',end='\n\n')
#     if i>5:
#         break
#     i=i+1
    
    
    
    #     neurons_list.remove(item)
#     try:
#         res = generate_test(w_net,b_net,layer_no,neuron_no,time_steps)
#         end_time = time.time()
#         print("Time for test generation: ",end_time-start_time)
#         use_generated_test(res)


#         if item in neurons_list:
#             neurons_list.remove(item)
#         print(len(neurons_list),' neurons remaining...',end='\n\n')
#     #     neurons_list.remove(item)
#     except Exception as e:
#         continue

In [34]:
import time
unknown_list=[]
spiked_list=[]
dead_list=[]
unknown=0
i=0
time_steps=1
print("Generating test cases for the candidate dead neurons:",end="\n\n")
start=time.time()
index = 0
while index < len(neurons_list):
    item=neurons_list[0]
    layer_no = int(item.split('_')[0][1:])
    neuron_no = int(item.split('_')[1])
    
    print(item," : ",end="")
    start_time = time.time()
    res = generate_test(w_net,b_net,layer_no,neuron_no,1)
    end_time = time.time()
    print("Time for test generation: ",end_time-start_time)
    
    if res==0:
        dead_list.append(item)
        if item in neurons_list:
            neurons_list.remove(item)
    elif res==1:
        unknown_list.append(item)
#         if item in neurons_list:
#             neurons_list.remove(item)
        unknown=unknown+1
        if(unknown>1):
            break
    else:
        spiked_list.append(item)
        use_generated_test(res)
#         if item in neurons_list:
#             neurons_list.remove(item)
#     else:
#         print('Error')
    
    print(len(neurons_list),' neurons remaining...',end='\n\n')

    

Generating test cases for the candidate dead neurons:

X1_2  : Has spiked
Time for test generation:  15.860190391540527
X1_2 X1_156 X3_44 46  neurons remaining...

X1_21  : Has spiked
Time for test generation:  15.819727659225464
X1_21 X1_146 X1_167 43  neurons remaining...

X1_26  : Has spiked
Time for test generation:  15.78456163406372
X1_26 X1_39 X1_117 X1_126 X1_192 X1_195 X1_196 X4_10 35  neurons remaining...

X1_28  : Has spiked
Time for test generation:  15.74194598197937
X1_28 X1_131 X1_135 X1_178 X3_38 30  neurons remaining...

X1_31  : Has spiked
Time for test generation:  15.814618825912476
X1_31 29  neurons remaining...

X1_44  : Has spiked
Time for test generation:  15.772465944290161
X1_44 28  neurons remaining...

X1_46  : Has spiked
Time for test generation:  15.784194946289062
X1_46 X1_105 26  neurons remaining...

X1_55  : Has spiked
Time for test generation:  15.85608720779419
X1_55 X1_75 X4_2 23  neurons remaining...

X1_68  : Has spiked
Time for test generation:  

In [35]:
# remaining_list

## A. Upto Layer 1

In [39]:
time_steps=1
import time
# remaining_list=[]
neurons_list_2 = [element for element in neurons_list if element.startswith("X2_")]
i=0
print("Generating test cases for the candidate dead neurons:",end="\n\n")
start=time.time()
index = 0
while index < len(neurons_list_2):
    item=neurons_list_2[0]
    layer_no = int(item.split('_')[0][1:])
    neuron_no = int(item.split('_')[1])
    weights,biases=find_params(net)
    no=neuron_no
    wc2=abstraction(weights,biases,2,no)
    num_hidden1 = len(wc2[0]) 
    num_hidden2 = len(wc2[1])
#     net_abs=Net().to(device)
    num_hidden = [num_hidden1,num_hidden2,50]
    net_abs=Net()
    net_abs.fc1.bias.data=torch.zeros(len(wc2[0]), dtype=torch.float32)
    net_abs.fc2.bias.data=torch.zeros(len(wc2[1]), dtype=torch.float32)
    net_abs.fc1.weight.data = torch.tensor(np.array(wc2[0]), dtype=torch.float32)
    net_abs.fc2.weight.data = torch.tensor(np.array(wc2[1]), dtype=torch.float32)

    w_abs,b_abs=find_params(net_abs)
#     print(net_abs)


    print(item," : ",end="")
    start_time = time.time()
    res = generate_test(w_abs,b_abs,layer_no,0,time_steps)
    end_time = time.time()
    print("Time for test generation: ",end_time-start_time)
#     if res!=1:
#         use_generated_test(res)
#     else:
#         remaining_list.append(item)
    if res==0:
        dead_list.append(item)
        if item in neurons_list:
            neurons_list.remove(item)
    elif res==1:
        unknown_list.append(item)
#         if item in neurons_list:
#             neurons_list.remove(item)
        unknown=unknown+1
        if(unknown>1):
            break
    else:
        print('Trying to fire others...')
        spiked_list.append(item)
        use_generated_test(res)
        if item in neurons_list:
            neurons_list.remove(item)


    if item in neurons_list:
        neurons_list.remove(item)
    if item in neurons_list_2:
        neurons_list_2.remove(item)
    print(len(neurons_list),' neurons remaining...',end='\n\n')
#     if i>5:
#         break
#     i=i+1

Generating test cases for the candidate dead neurons:

Layer 1 : 200
Layer 1 : 200
Layer 1 : 200
Layer 1 : 2
X2_88  : Has spiked
Time for test generation:  0.3223114013671875
Trying to fire others...
1  neurons remaining...



In [36]:
# weights,biases=find_params(net)
# no=3
# wc2=abstraction(weights,biases,2,no)
# num_hidden1 = len(wc2[0]) 
# num_hidden2 = len(wc2[1])
# net_abs=Net().to(device)
# num_hidden = [num_hidden1,num_hidden2,50]
# net_abs=Net()
# net_abs.fc1.bias.data=torch.zeros(len(wc2[0]), dtype=torch.float32)
# net_abs.fc2.bias.data=torch.zeros(len(wc2[1]), dtype=torch.float32)
# net_abs.fc1.weight.data = torch.tensor(np.array(wc2[0]), dtype=torch.float32)
# net_abs.fc2.weight.data = torch.tensor(np.array(wc2[1]), dtype=torch.float32)

# w_abs,b_abs=find_params(net_abs)

# net_abs

Layer 1 : 200
Layer 1 : 200
Layer 1 : 200
Layer 1 : 2


Net(
  (fc1): Linear(in_features=784, out_features=2, bias=True)
  (lif1): Leaky()
  (fc2): Linear(in_features=2, out_features=1, bias=True)
  (lif2): Leaky()
  (fc3): Linear(in_features=1, out_features=50, bias=True)
  (lif3): Leaky()
  (fc4): Linear(in_features=50, out_features=10, bias=True)
  (lif4): Leaky()
)

In [37]:
# weights,biases=find_params(net)

In [1]:
# wc2=abstraction(weights,biases,2)

In [ ]:
# len(wc2)

In [ ]:
# num_hidden1 = len(wc2[0]) 
# num_hidden2 = len(wc2[1])
# net_abs=Net().to(device)

In [ ]:
# num_hidden1

In [ ]:
# num_hidden2

In [ ]:
# num_hidden = [num_hidden1,num_hidden2,50]

In [ ]:
# num_hidden

In [ ]:
# net_abs=Net()
# net_abs

In [ ]:
# net_abs.fc1.bias.data=torch.zeros(len(wc2[0]), dtype=torch.float32)
# net_abs.fc2.bias.data=torch.zeros(len(wc2[1]), dtype=torch.float32)
# # net_abs.fc2.bias.data=torch.zeros(10, dtype=torch.float32)

# # net_abs.fc3.bias.data=torch.zeros(len(wc2[2]), dtype=torch.float32)
# # net_abs.fc4.bias.data=torch.zeros(len(wc2[3]), dtype=torch.float32)
# # net_abs.fc5.bias.data=torch.zeros(10, dtype=torch.float32)

# net_abs.fc1.weight.data = torch.tensor(np.array(wc2[0]), dtype=torch.float32)
# net_abs.fc2.weight.data = torch.tensor(np.array(wc2[1]), dtype=torch.float32)

# # net_abs.fc3.weight.data = torch.tensor(np.array(wc2[2]), dtype=torch.float32)
# # net_abs.fc4.weight.data = torch.tensor(np.array(wc2[3]), dtype=torch.float32)
# # net_abs.fc5.weight.data = torch.tensor(np.array(wc2[4]), dtype=torch.float32)

In [ ]:
# w_abs,b_abs=find_params(net_abs)

In [ ]:
# import time
# # remaining_list=[]
# neurons_list_2 = [element for element in neurons_list if element.startswith("X2_")]
# i=0
# print("Generating test cases for the candidate dead neurons:",end="\n\n")
# start=time.time()
# index = 0
# while index < len(neurons_list_2):
#     item=neurons_list_2[0]
#     layer_no = int(item.split('_')[0][1:])
#     neuron_no = int(item.split('_')[1])
#     weights,biases=find_params(net)
#     no=neuron_no
#     wc2=abstraction(weights,biases,2,no)
#     num_hidden1 = len(wc2[0]) 
#     num_hidden2 = len(wc2[1])
#     net_abs=Net().to(device)
#     num_hidden = [num_hidden1,num_hidden2,50]
#     net_abs=Net()
#     net_abs.fc1.bias.data=torch.zeros(len(wc2[0]), dtype=torch.float32)
#     net_abs.fc2.bias.data=torch.zeros(len(wc2[1]), dtype=torch.float32)
#     net_abs.fc1.weight.data = torch.tensor(np.array(wc2[0]), dtype=torch.float32)
#     net_abs.fc2.weight.data = torch.tensor(np.array(wc2[1]), dtype=torch.float32)

#     w_abs,b_abs=find_params(net_abs)
#     print(net_abs)


#     print(item," : ",end="")
#     start_time = time.time()
#     res = generate_test(w_abs,b_abs,layer_no,0,1)
#     end_time = time.time()
#     print("Time for test generation: ",end_time-start_time)
#     if res!=1:
#         use_generated_test(res)
#     else:
#         remaining_list.append(item)


#     if item in neurons_list:
#         neurons_list.remove(item)
#     if item in neurons_list_2:
#         neurons_list_2.remove(item)
#     print(len(neurons_list),' neurons remaining...',end='\n\n')
#     if i>5:
#         break
#     i=i+1

In [ ]:
# res = generate_test(w_abs,b_abs,2,3,1)

## B. Upto Layer 2

In [40]:
time_steps=1
import time
# remaining_list=[]
neurons_list_3 = [element for element in neurons_list if element.startswith("X3_")]
i=0
print("Generating test cases for the candidate dead neurons:",end="\n\n")
start=time.time()
index = 0
while index < len(neurons_list_3):
    item=neurons_list_3[0]
    layer_no = int(item.split('_')[0][1:])
    neuron_no = int(item.split('_')[1])
    weights,biases=find_params(net)
    no=neuron_no
    wc2=abstraction(weights,biases,3,no)
    num_hidden1 = len(wc2[0]) 
    num_hidden2 = len(wc2[1])
    num_hidden2 = len(wc2[2])

    #     net_abs=Net().to(device)
    num_hidden = [num_hidden1,num_hidden2,50]
    net_abs=Net()
    net_abs.fc1.bias.data=torch.zeros(len(wc2[0]), dtype=torch.float32)
    net_abs.fc2.bias.data=torch.zeros(len(wc2[1]), dtype=torch.float32)
    net_abs.fc3.bias.data=torch.zeros(len(wc2[2]), dtype=torch.float32)

    
    net_abs.fc1.weight.data = torch.tensor(np.array(wc2[0]), dtype=torch.float32)
    net_abs.fc2.weight.data = torch.tensor(np.array(wc2[1]), dtype=torch.float32)
    net_abs.fc3.weight.data = torch.tensor(np.array(wc2[2]), dtype=torch.float32)

    w_abs,b_abs=find_params(net_abs)
#     print(net_abs)


    print(item," : ",end="")
    start_time = time.time()
    res = generate_test(w_abs,b_abs,layer_no,0,time_steps)
    end_time = time.time()
    print("Time for test generation: ",end_time-start_time)
#     if res!=1:
#         use_generated_test(res)
#     else:
#         remaining_list.append(item)
    if res==0:
        dead_list.append(item)
        if item in neurons_list:
            neurons_list.remove(item)
    elif res==1:
        unknown_list.append(item)
#         if item in neurons_list:
#             neurons_list.remove(item)
        unknown=unknown+1
        if(unknown>1):
            break
    else:
        print('Trying to fire others...')
        spiked_list.append(item)
        use_generated_test(res)
        if item in neurons_list:
            neurons_list.remove(item)


    if item in neurons_list:
        neurons_list.remove(item)
    if item in neurons_list_3:
        neurons_list_3.remove(item)
    print(len(neurons_list),' neurons remaining...',end='\n\n')
#     if i>5:
#         break
#     i=i+1

Generating test cases for the candidate dead neurons:



In [ ]:
# weights,biases=find_params(net)

In [ ]:
# wc2=abstraction(weights,biases,3)

In [ ]:
# num_hidden1 = len(wc2[0]) 
# num_hidden2 = len(wc2[1])
# # num_hidden2 = len(wc2[1])

In [ ]:
# num_hidden = [num_hidden1,num_hidden2,50]

In [ ]:
# net_abs=Net().to(device)
# net_abs

In [ ]:
# net_abs.fc1.bias.data=torch.zeros(len(wc2[0]), dtype=torch.float32)
# net_abs.fc2.bias.data=torch.zeros(len(wc2[1]), dtype=torch.float32)
# # net_abs.fc3.bias.data=torch.zeros(10, dtype=torch.float32)

# net_abs.fc3.bias.data=torch.zeros(len(wc2[2]), dtype=torch.float32)
# # net_abs.fc4.bias.data=torch.zeros(len(wc2[3]), dtype=torch.float32)
# # net_abs.fc5.bias.data=torch.zeros(10, dtype=torch.float32)

# net_abs.fc1.weight.data = torch.tensor(np.array(wc2[0]), dtype=torch.float32)
# net_abs.fc2.weight.data = torch.tensor(np.array(wc2[1]), dtype=torch.float32)
# net_abs.fc3.weight.data = torch.tensor(np.array(wc2[2]), dtype=torch.float32)
# # net_abs.fc4.weight.data = torch.tensor(np.array(wc2[3]), dtype=torch.float32)
# # net_abs.fc5.weight.data = torch.tensor(np.array(wc2[4]), dtype=torch.float32)

In [ ]:
# w_abs,b_abs=find_params(net_abs)

In [ ]:
# w_abs

In [ ]:
# neurons_list=['X1_3', 'X1_6', 'X1_10', 'X1_12', 'X1_13', 'X1_14', 'X1_17', 'X1_18', 'X1_19', 'X1_21', 'X1_22', 'X1_27', 'X1_30', 'X1_35', 'X1_39', 'X1_40', 'X1_43', 'X1_45', 'X1_46', 'X1_49', 'X1_51', 'X1_52', 'X1_53', 'X1_55', 'X1_56', 'X1_62', 'X1_67', 'X1_68', 'X1_70', 'X1_71', 'X1_85', 'X1_86', 'X1_95', 'X1_99', 'X1_100', 'X1_106', 'X1_109', 'X1_111', 'X1_114', 'X1_115', 'X1_120', 'X1_121', 'X1_128', 'X1_132', 'X1_138', 'X1_140', 'X1_142', 'X1_144', 'X1_147', 'X1_150', 'X1_152', 'X1_159', 'X1_164', 'X1_167', 'X1_171', 'X1_175', 'X1_176', 'X1_178', 'X1_179', 'X1_188', 'X1_189', 'X1_192', 'X1_194', 'X2_2', 'X2_3', 'X2_7', 'X2_8', 'X2_9', 'X2_14', 'X2_16', 'X2_17', 'X2_18', 'X2_19', 'X2_23', 'X2_25', 'X2_26', 'X2_27', 'X2_28', 'X2_29', 'X2_32', 'X2_33', 'X2_35', 'X2_39', 'X2_40', 'X2_47', 'X2_48', 'X2_50', 'X2_52', 'X2_54', 'X2_55', 'X2_57', 'X2_60', 'X2_61', 'X2_64', 'X2_65', 'X2_66', 'X2_67', 'X2_70', 'X2_72', 'X2_74', 'X2_75', 'X2_76', 'X2_79', 'X2_80', 'X2_81', 'X2_82', 'X2_84', 'X2_88', 'X2_89', 'X2_91', 'X2_93', 'X2_97', 'X2_98', 'X2_99', 'X2_100', 'X3_2', 'X3_4', 'X3_6', 'X3_8', 'X3_10', 'X3_12', 'X3_13', 'X3_15', 'X3_16', 'X3_17', 'X3_20', 'X3_22', 'X3_23', 'X3_24', 'X3_25', 'X3_26', 'X3_27', 'X3_28', 'X3_29', 'X3_31', 'X3_32', 'X3_33', 'X3_34', 'X3_36', 'X3_37', 'X3_38', 'X3_39', 'X3_41', 'X3_43', 'X3_45', 'X3_47', 'X3_48', 'X4_4']

In [ ]:
# import time
# # remaining_list=[]
# neurons_list_3 = [element for element in neurons_list if element.startswith("X3_")]
# i=0
# print("Generating test cases for the candidate dead neurons:",end="\n\n")
# start=time.time()
# index = 0
# while index < len(neurons_list_3):
#     item=neurons_list_3[0]
#     layer_no = int(item.split('_')[0][1:])
#     neuron_no = int(item.split('_')[1])
    
#     print(item," : ",end="")
#     start_time = time.time()
#     res = generate_test(w_abs,b_abs,layer_no,neuron_no,1)
#     end_time = time.time()
#     print("Time for test generation: ",end_time-start_time)
#     if res!=1:
#         pass
# #         use_generated_test(res)
#     else:
#         remaining_list.append(item)


#     if item in neurons_list_3:
#         neurons_list_3.remove(item)
#     if item in neurons_list:
#         neurons_list.remove(item)
#     print(len(neurons_list_3),' neurons remaining...',end='\n\n')
#     if i>5:
#         break
#     i=i+1

In [ ]:
# res = generate_test(w_abs,b_abs,4,10,1)

## C. Upto Layer 3

In [42]:
time_steps=1
import time
# remaining_list=[]
neurons_list_4 = [element for element in neurons_list if element.startswith("X4_")]
i=0
print("Generating test cases for the candidate dead neurons:",end="\n\n")
start=time.time()
index = 0
while index < len(neurons_list_4):
    item=neurons_list_4[0]
    layer_no = int(item.split('_')[0][1:])
    neuron_no = int(item.split('_')[1])
    weights,biases=find_params(net)
    no=neuron_no
    wc2=abstraction(weights,biases,4,no)
    num_hidden1 = len(wc2[0]) 
    num_hidden2 = len(wc2[1])
    num_hidden3 = len(wc2[2])

    #     net_abs=Net().to(device)
    num_hidden = [num_hidden1,num_hidden2,num_hidden3]
    net_abs=Net()
    net_abs.fc1.bias.data=torch.zeros(len(wc2[0]), dtype=torch.float32)
    net_abs.fc2.bias.data=torch.zeros(len(wc2[1]), dtype=torch.float32)
    net_abs.fc3.bias.data=torch.zeros(len(wc2[2]), dtype=torch.float32)
    net_abs.fc4.bias.data=torch.zeros(len(wc2[3]), dtype=torch.float32)
    

    
    net_abs.fc1.weight.data = torch.tensor(np.array(wc2[0]), dtype=torch.float32)
    net_abs.fc2.weight.data = torch.tensor(np.array(wc2[1]), dtype=torch.float32)
    net_abs.fc3.weight.data = torch.tensor(np.array(wc2[2]), dtype=torch.float32)
    net_abs.fc4.weight.data = torch.tensor(np.array(wc2[3]), dtype=torch.float32)

    w_abs,b_abs=find_params(net_abs)
#     print(net_abs)


    print(item," : ",end="")
    start_time = time.time()
    res = generate_test(w_abs,b_abs,layer_no,0,time_steps)
    end_time = time.time()
    print("Time for test generation: ",end_time-start_time)
#     if res!=1:
#         use_generated_test(res)
#     else:
#         remaining_list.append(item)
    if res==0:
        dead_list.append(item)
        if item in neurons_list:
            neurons_list.remove(item)
    elif res==1:
        unknown_list.append(item)
#         if item in neurons_list:
#             neurons_list.remove(item)
        unknown=unknown+1
        if(unknown>1):
            break
    else:
        print('Trying to fire others...')
        spiked_list.append(item)
        use_generated_test(res)
        if item in neurons_list:
            neurons_list.remove(item)


    if item in neurons_list:
        neurons_list.remove(item)
    if item in neurons_list_4:
        neurons_list_4.remove(item)
    print(len(neurons_list),' neurons remaining...',end='\n\n')
#     if i>5:
#         break
#     i=i+1

Generating test cases for the candidate dead neurons:

Layer 1 : 200
Layer 2 : 100
Layer 3 : 50
Layer 1 : 400
Layer 2 : 200
Layer 3 : 50
Layer 1 : 800
Layer 2 : 400
Layer 3 : 50
Layer 1 : 4
Layer 2 : 4
Layer 3 : 2
X4_5  : Has spiked
Time for test generation:  0.41115236282348633
Trying to fire others...
0  neurons remaining...



In [ ]:
weights,biases=find_params(net)

In [ ]:
wc2=abstraction(weights,biases,4)

In [ ]:
len(wc2)

In [ ]:
num_hidden1 = len(wc2[0]) 
num_hidden2 = len(wc2[1])
num_hidden3 = len(wc2[2])

In [ ]:
num_hidden = [num_hidden1,num_hidden2,num_hidden3]

In [ ]:
num_hidden

In [ ]:
net_abs=Net().to(device)
net_abs

In [ ]:
net_abs.fc1.bias.data=torch.zeros(len(wc2[0]), dtype=torch.float32)
net_abs.fc2.bias.data=torch.zeros(len(wc2[1]), dtype=torch.float32)
# net_abs.fc3.bias.data=torch.zeros(10, dtype=torch.float32)

net_abs.fc3.bias.data=torch.zeros(len(wc2[2]), dtype=torch.float32)
net_abs.fc4.bias.data=torch.zeros(len(wc2[3]), dtype=torch.float32)
# net_abs.fc5.bias.data=torch.zeros(10, dtype=torch.float32)

net_abs.fc1.weight.data = torch.tensor(np.array(wc2[0]), dtype=torch.float32)
net_abs.fc2.weight.data = torch.tensor(np.array(wc2[1]), dtype=torch.float32)
net_abs.fc3.weight.data = torch.tensor(np.array(wc2[2]), dtype=torch.float32)
net_abs.fc4.weight.data = torch.tensor(np.array(wc2[3]), dtype=torch.float32)
# net_abs.fc5.weight.data = torch.tensor(np.array(wc2[4]), dtype=torch.float32)

In [ ]:
w_abs,b_abs=find_params(net_abs)

In [ ]:
neurons_list=['X4_5','X4_10']

In [ ]:
import time
# remaining_list=[]
neurons_list_4 = [element for element in neurons_list if element.startswith("X4_")]
i=0
print("Generating test cases for the candidate dead neurons:",end="\n\n")
start=time.time()
index = 0
while index < len(neurons_list_4):
    item=neurons_list_4[0]
    layer_no = int(item.split('_')[0][1:])
    neuron_no = int(item.split('_')[1])
    
    print(item," : ",end="")
    start_time = time.time()
    res = generate_test(w_abs,b_abs,layer_no,neuron_no,1)
    end_time = time.time()
    print("Time for test generation: ",end_time-start_time)
    if res!=1:
        use_generated_test(res)
    else:
        remaining_list.append(item)


    if item in neurons_list_4:
        neurons_list_4.remove(item)
    if item in neurons_list:
        neurons_list.remove(item)
    print(len(neurons_list_3),' neurons remaining...',end='\n\n')
    if i>5:
        break
    i=i+1